<center>

# Solución de proyecto

---

</center>

**Cargar librerias**

In [1]:
import opendatasets as od
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.svm import SVC

**Cargar dataset**

In [2]:
first_path = os.getcwd()

In [3]:
os.chdir(first_path)
dataset_link = "https://www.kaggle.com/competitions/udea-ai-4-eng-20252-pruebas-saber-pro-colombia/overview"
old_name = "udea-ai-4-eng-20252-pruebas-saber-pro-colombia"
new_name = "data"

# ejecutar si la carpeta "data" no existe
if not os.path.exists(new_name):
    od.download(dataset_link)
    if os.path.exists(old_name):
        os.rename(old_name, new_name)
    else:
        print(f"No se encontró la carpeta {old_name} después de la descarga")
else:
    print(f"La carpeta '{new_name}' ya existe")


La carpeta 'data' ya existe


In [4]:
# moverse a la carpeta
os.chdir(new_name)
print(os.listdir())
archivo = "train.csv"
data = pd.read_csv(archivo)
os.chdir("../") # volvemos a la carpeta de todo el repositorio

['submission_example.csv', 'test.csv', 'train.csv']


Una revision rápida para ver los datos al inicio

In [5]:
data.head()

,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,...,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,...,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 692500 entries, 0 to 692499
Data columns (total 21 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   ID                           692500 non-null  int64  
 1   PERIODO_ACADEMICO            692500 non-null  int64  
 2   E_PRGM_ACADEMICO             692500 non-null  object 
 3   E_PRGM_DEPARTAMENTO          692500 non-null  object 
 4   E_VALORMATRICULAUNIVERSIDAD  686213 non-null  object 
 5   E_HORASSEMANATRABAJA         661643 non-null  object 
 6   F_ESTRATOVIVIENDA            660363 non-null  object 
 7   F_TIENEINTERNET              665871 non-null  object 
 8   F_EDUCACIONPADRE             669322 non-null  object 
 9   F_TIENELAVADORA              652727 non-null  object 
 10  F_TIENEAUTOMOVIL             648877 non-null  object 
 11  E_PRIVADO_LIBERTAD           692500 non-null  object 
 12  E_PAGOMATRICULAPROPIO        686002 non-null  object 
 13 

In [7]:
data.describe(include="all")

,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,...,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
count,692500.000000,692500.000000,692500,692500,686213,661643,660363,665871,669322,652727,...,692500,686002,654397,665871,668836,692500,692500.000000,692500.000000,692500.000000,692500.000000
unique,NaN,NaN,948,31,8,5,7,2,12,2,...,2,2,2,2,12,4,NaN,NaN,NaN,NaN
top,NaN,NaN,DERECHO,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,Más de 30 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,NaN,NaN,NaN,NaN
freq,NaN,NaN,53244,282159,204048,249352,232671,592514,128289,563390,...,692466,382201,597670,592514,141744,175619,NaN,NaN,NaN,NaN
mean,494606.130576,20198.366679,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.268629,0.259996,0.262087,0.262903
std,285585.209455,10.535037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.122130,0.093480,0.058862,0.067944
min,1.000000,20183.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000
25%,247324.750000,20195.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.203000,0.212000,0.254000,0.255000
50%,494564.500000,20195.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.240000,0.271000,0.276000,0.285000
75%,741782.500000,20203.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.314000,0.309000,0.293000,0.303000


**Cuando se realizó el preprocesado se obtuvo un dataset mas limpio que el original, por lo que se usará este para la solucion**

En este caso no nos desharemos de "ID" ya que es util para la participación en kaggle

In [8]:
def limpieza(datos):

    # Eliminar columna redundante
    data = datos.drop(columns=["F_TIENEINTERNET.1"])

    #  Rellenar NaN con la moda de cada columna
    col_nan = data.columns[data.isna().any()]
    for col in col_nan:
        moda = data[col].mode()[0]
        data[col] = data[col].fillna(moda)

    #One-hot de las variables binarias (Si/No)
    data = pd.get_dummies(
        data,
        columns=[
            "F_TIENEINTERNET",
            "F_TIENELAVADORA",
            "E_PAGOMATRICULAPROPIO",
            "F_TIENEAUTOMOVIL",
            "E_PRIVADO_LIBERTAD",
            "F_TIENECOMPUTADOR",
        ],
        drop_first=True,
        dummy_na=False,
        dtype="int8"
    )

    # Estrato como variable ordinal
    cambios_estrato = {'Sin Estrato': 0,'Estrato 1': 1,'Estrato 2': 2,'Estrato 3': 3,'Estrato 4': 4,'Estrato 5': 5,'Estrato 6': 6}
    data["F_ESTRATOVIVIENDA"] = data["F_ESTRATOVIVIENDA"].map(cambios_estrato).astype("int8")
    data.columns = data.columns.str.replace(r'_(?:S[ií]|S)?$', '', regex=True)

    # Limpiar sintaxis 
    col_fix = ["E_PRGM_ACADEMICO", "E_PRGM_DEPARTAMENTO","F_EDUCACIONPADRE", "F_EDUCACIONMADRE"]
    for c in col_fix:
        s = data[c].astype("string")
        s = (s.str.normalize("NFKD").str.encode("ascii", "ignore").str.decode("utf-8").str.lower().str.replace(r"[^a-z0-9 ]", " ", regex=True).str.replace(r"\s+", " ", regex=True).str.strip())
        # pasar de "de" a "en"
        s = s.str.replace(r"\bde\b", "en", regex=True)
        data[c] = s

    # Limpieza de E_PRGM_ACADEMICO
    col_prog = "E_PRGM_ACADEMICO"
    reemplazos = {r"(?i)\bcomunicacion socialy periodismo\b": "comunicacion social y periodismo",r"(?i)\bcomunicacion socialyperiodismo\b": "comunicacion social y periodismo",r"(?i)\bcomunicacion social\s+periodismo\b": "comunicacion social y periodismo",r"(?i)\bcontaduria pblica\b": "contaduria publica",}
    for pat, rep in reemplazos.items():
        data[col_prog] = data[col_prog].str.replace(pat, rep, regex=True)
    data[col_prog] = data[col_prog].str.replace(r"\s+", " ", regex=True).str.strip()

    #Agrupar programas poco frecuentes en "otros"
    conteos = data[col_prog].value_counts()
    raros = conteos[conteos <= 6925].index
    data[col_prog] = data[col_prog].replace(raros, "otros")

    # Matricula universidad como ordinal
    cambios_matricula = {'No pagó matrícula': 0,'Menos de 500 mil': 1,'Entre 500 mil y menos de 1 millón': 2,'Entre 1 millón y menos de 2.5 millones': 3,'Entre 2.5 millones y menos de 4 millones': 4,'Entre 4 millones y menos de 5.5 millones': 5,'Entre 5.5 millones y menos de 7 millones': 6,'Más de 7 millones': 7,}
    data["E_VALORMATRICULAUNIVERSIDAD"] = (data["E_VALORMATRICULAUNIVERSIDAD"].map(cambios_matricula).astype("int8"))

    #Horas que trabaja a la semana como ordinal
    cambios_horas = {'0': 0,'Menos de 10 horas': 1,'Entre 11 y 20 horas': 2,'Entre 21 y 30 horas': 3,'Más de 30 horas': 4,}
    data["E_HORASSEMANATRABAJA"] = (data["E_HORASSEMANATRABAJA"].map(cambios_horas).astype("int8"))

    # Nivel educativo de padre/madre como ordinal
    cambios_edu = {'ninguno': 0,'primaria incompleta': 1,'primaria completa': 2,'secundaria bachillerato incompleta': 3,'secundaria bachillerato completa': 4,'tecnica o tecnologica incompleta': 5,'tecnica o tecnologica completa': 6,'educacion profesional incompleta': 7,'educacion profesional completa': 8,'postgrado': 9,'no sabe': 10,'no aplica': 11,}
    data["F_EDUCACIONPADRE"] = data["F_EDUCACIONPADRE"].map(cambios_edu).astype("int8")
    data["F_EDUCACIONMADRE"] = data["F_EDUCACIONMADRE"].map(cambios_edu).astype("int8")

    # One-hot de programa académico y departamento
    data = pd.get_dummies(data,columns=["E_PRGM_ACADEMICO", "E_PRGM_DEPARTAMENTO"],drop_first=True,dummy_na=False,dtype="int8")

    return data


Ahora obtengamos los datos de **entrenamiento** y de **prueba**

In [9]:
data_train = data
# moverse a la carpeta
os.chdir(new_name)
print(os.listdir())
archivo = "test.csv"
data_test = pd.read_csv(archivo)
os.chdir("../") # volvemos a la carpeta de todo el repositorio

['submission_example.csv', 'test.csv', 'train.csv']


In [10]:
data_test.shape

(296786, 20)

In [11]:
data_train = limpieza(data_train)
data_test  = limpieza(data_test)

# mapear la variable objetivo SOLO en train
# convertir RENDIMIENTO_GLOBAL a números (solo existe en train)
cambios_rg = {'bajo': 0, 'medio-bajo': 1, 'medio-alto': 2, 'alto': 3}
data_train["RENDIMIENTO_GLOBAL"] = (
    data_train["RENDIMIENTO_GLOBAL"].map(cambios_rg).astype("int8")
)

ahora una vista rápida antes de continuar

In [12]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 692500 entries, 0 to 692499
Data columns (total 66 columns):
 #   Column                                                Non-Null Count   Dtype  
---  ------                                                --------------   -----  
 0   ID                                                    692500 non-null  int64  
 1   PERIODO_ACADEMICO                                     692500 non-null  int64  
 2   E_VALORMATRICULAUNIVERSIDAD                           692500 non-null  int8   
 3   E_HORASSEMANATRABAJA                                  692500 non-null  int8   
 4   F_ESTRATOVIVIENDA                                     692500 non-null  int8   
 5   F_EDUCACIONPADRE                                      692500 non-null  int8   
 6   F_EDUCACIONMADRE                                      692500 non-null  int8   
 7   RENDIMIENTO_GLOBAL                                    692500 non-null  int8   
 8   INDICADOR_1                                 

In [13]:
data_train.head()

,ID,PERIODO_ACADEMICO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_EDUCACIONPADRE,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,...,E_PRGM_DEPARTAMENTO_norte santander,E_PRGM_DEPARTAMENTO_putumayo,E_PRGM_DEPARTAMENTO_quindio,E_PRGM_DEPARTAMENTO_risaralda,E_PRGM_DEPARTAMENTO_san andres,E_PRGM_DEPARTAMENTO_santander,E_PRGM_DEPARTAMENTO_sucre,E_PRGM_DEPARTAMENTO_tolima,E_PRGM_DEPARTAMENTO_valle,E_PRGM_DEPARTAMENTO_vaupes
0,904256,20212,6,1,3,5,9,2,0.322,0.208,...,0,0,0,0,0,0,0,0,0,0
1,645256,20212,4,0,3,6,5,0,0.311,0.215,...,0,0,0,0,0,0,0,0,0,0
2,308367,20203,4,4,3,4,4,0,0.297,0.214,...,0,0,0,0,0,0,0,0,0,0
3,470353,20195,5,0,4,10,4,3,0.485,0.172,...,0,0,0,0,0,1,0,0,0,0
4,989032,20212,4,3,3,2,2,1,0.316,0.232,...,0,0,0,0,0,0,0,0,0,0


In [14]:
data_train.describe()

,ID,PERIODO_ACADEMICO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_EDUCACIONPADRE,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,...,E_PRGM_DEPARTAMENTO_norte santander,E_PRGM_DEPARTAMENTO_putumayo,E_PRGM_DEPARTAMENTO_quindio,E_PRGM_DEPARTAMENTO_risaralda,E_PRGM_DEPARTAMENTO_san andres,E_PRGM_DEPARTAMENTO_santander,E_PRGM_DEPARTAMENTO_sucre,E_PRGM_DEPARTAMENTO_tolima,E_PRGM_DEPARTAMENTO_valle,E_PRGM_DEPARTAMENTO_vaupes
count,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,...,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000
mean,494606.130576,20198.366679,3.514966,2.480606,2.497311,4.431242,4.474563,1.505227,0.268629,0.259996,...,0.032618,0.001148,0.011883,0.018309,0.000014,0.041629,0.012526,0.017214,0.064387,0.000020
std,285585.209455,10.535037,1.795653,1.523329,1.101868,2.856832,2.573949,1.121061,0.122130,0.093480,...,0.177635,0.033863,0.108360,0.134067,0.003800,0.199740,0.111215,0.130070,0.245441,0.004496
min,1.000000,20183.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,247324.750000,20195.000000,2.000000,1.000000,2.000000,2.000000,3.000000,1.000000,0.203000,0.212000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,494564.500000,20195.000000,3.000000,3.000000,2.000000,4.000000,4.000000,2.000000,0.240000,0.271000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,741782.500000,20203.000000,5.000000,4.000000,3.000000,7.000000,6.000000,3.000000,0.314000,0.309000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,989286.000000,20213.000000,7.000000,4.000000,6.000000,11.000000,11.000000,3.000000,0.657000,0.487000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


Ahora vamos a obtener los x, y

In [15]:
# columnas de features (todas menos la etiqueta)
feature_cols = [c for c in data_train.columns if c != "RENDIMIENTO_GLOBAL"]

# Crear en test las columnas que esten en train y falten en test
missing_in_test = set(feature_cols) - set(data_test.columns)
for c in missing_in_test:
    data_test[c] = 0

# Quitar en test las columnas que sobren es decir, que no esten en train 
extra_in_test = set(data_test.columns) - set(feature_cols)
if extra_in_test:
    data_test = data_test.drop(columns=list(extra_in_test))

# Ordenar columnas de test igual que en train
data_test = data_test[feature_cols]

# Matrices finales para el modelo
Xtrain = data_train[feature_cols].values
Ytrain = data_train["RENDIMIENTO_GLOBAL"].values
Xtest  = data_test[feature_cols].values


In [16]:
print (Xtrain.shape, Ytrain.shape)
print (Xtest.shape)

(692500, 65) (692500,)
(296786, 65)


In [17]:
#  TOMAR UNA MUESTRA DEL TRAIN


X_sub, _, y_sub, _ = train_test_split(Xtrain, Ytrain,train_size=50000,stratify=Ytrain,random_state=0)

print("Tamaño de la muestra:", X_sub.shape)


# evaluar modelos


# Naive Bayes (rápido)
nb = BernoulliNB()
print("BernoulliNB:", cross_val_score(nb, X_sub, y_sub, cv=2))

# Regresión Logística
logreg = LogisticRegression(max_iter=500,n_jobs=-1,solver="saga")
print("LogisticRegression:", cross_val_score(logreg, X_sub, y_sub, cv=2))

# Random Forest
rf = RandomForestClassifier(n_estimators=100,max_depth=15,n_jobs=-1,random_state=0)
print("RandomForest:", cross_val_score(rf, X_sub, y_sub, cv=2))

# HistGradientBoosting
hgb = HistGradientBoostingClassifier(max_depth=10,random_state=0)
print("HistGradientBoosting:", cross_val_score(hgb, X_sub, y_sub, cv=2))

Tamaño de la muestra: (50000, 65)
BernoulliNB: [0.35996 0.36624]


c:\Users\HP\anaconda3\envs\tensorflow\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\HP\anaconda3\envs\tensorflow\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


LogisticRegression: [0.25268 0.25388]
RandomForest: [0.38904 0.39108]
HistGradientBoosting: [0.40236 0.4008 ]


In [18]:
hgb.fit(Xtrain, Ytrain)
preds_ts = hgb.predict(Xtest)
preds_ts

array([0, 2, 3, ..., 2, 3, 3], dtype=int8)

In [19]:
# Mapear a texto
inv_cambios = {0: "bajo", 1: "medio-bajo", 2: "medio-alto", 3: "alto"}
preds_ts_labels = pd.Series(preds_ts).map(inv_cambios)

# Usar preds_ts_labels en el submission
submission = pd.DataFrame([data_test.ID, pd.Series(preds_ts_labels, name="RENDIMIENTO_GLOBAL")]).T

submission.head()

submission.to_csv("kaggle_competition.csv", index=False)